# Backtest Parameter Grid (Local Jupyter)

This notebook runs parameter grid explorations over the trading strategy using the existing backtest engine, **for local Jupyter**.

Steps:
1. Start Jupyter from the project root (where `src/` lives).
2. Open this notebook from the `notebooks/` directory.
3. Run the cells from top to bottom.

# Initialize

In [1]:
import os
import sys

# Assume this notebook lives in '<project_root>/notebooks'.
# Move up one level to the project root and add it to sys.path so 'src' is importable.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if os.path.exists(os.path.join(project_root, 'src')):
    os.chdir(project_root)

if project_root not in sys.path:
    sys.path.append(project_root)

print('Working directory:', os.getcwd())
print('Has src?:', os.path.exists(os.path.join(os.getcwd(), 'src')))

Working directory: C:\Users\Anton\SRC\my\ml_lstm
Has src?: True


In [2]:
import itertools
import os

import numpy as np
import pandas as pd

from src.backtest import (
    _compute_atr_series,
    _compute_backtest_metrics,
    _make_naive_prediction_provider,
    _make_model_prediction_provider,
    _load_predictions_csv,
    _make_csv_prediction_provider,
)
from src.backtest_engine import BacktestConfig, run_backtest
from src.trading_strategy import StrategyConfig
from src.config import FREQUENCY, get_hourly_data_csv_path

In [3]:
# Configuration for this notebook run
frequency = '60min'   # or FREQUENCY
initial_equity = 10_000.0
prediction_mode = 'csv'  # 'naive', 'model', or 'csv'
predictions_csv = "backtests/nvda_60min_model_predictions_checkpoint.csv"     # set a path if using 'csv' mode

csv_path = get_hourly_data_csv_path(frequency)
print('Using OHLC data from:', csv_path)
data = pd.read_csv(csv_path)

# Basic sanity check
required_cols = {'Open', 'High', 'Low', 'Close'}
missing = required_cols - set(data.columns)
if missing:
    raise ValueError(f'Data file {csv_path} is missing required columns: {missing}')

# Compute ATR(14) and a scalar ATR proxy
atr_series = _compute_atr_series(data, window=14)
atr_like = float(atr_series.dropna().mean()) if not atr_series.dropna().empty else 1.0
print(f'Mean ATR proxy: {atr_like:.4f}')

# Build prediction provider
if prediction_mode == 'naive':
    provider = _make_naive_prediction_provider(offset_multiple=2.0, atr_like=atr_like)
elif prediction_mode == 'model':
    provider = _make_model_prediction_provider(data, frequency=frequency)
elif prediction_mode == 'csv':
    if not predictions_csv:
        raise ValueError('Set predictions_csv for csv mode')
    preds_df = _load_predictions_csv(predictions_csv)
    provider = _make_csv_prediction_provider(preds_df, data)
else:
    raise ValueError(f'Unknown prediction_mode: {prediction_mode}')

Using OHLC data from: C:\Users\Anton\SRC\my\ml_lstm\data\processed\nvda_60min.csv
Mean ATR proxy: 0.8272


C:\Users\Anton\SRC\my\ml_lstm\src\backtest.py:238: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = merged["predicted_price"].fillna(method="ffill").fillna(method="bfill").to_numpy()


In [4]:
def run_one(
    strat_cfg: StrategyConfig,
    commission_per_unit_per_leg: float = 0.005,
    min_commission_per_order: float = 1.0,
):
    """Run a single backtest with the given strategy and commission settings."""
    bt_cfg = BacktestConfig(
        initial_equity=initial_equity,
        strategy_config=strat_cfg,
        model_error_sigma=atr_like,
        fixed_atr=atr_like,
        commission_per_unit_per_leg=commission_per_unit_per_leg,
        min_commission_per_order=min_commission_per_order,
    )

    result = run_backtest(
        data=data,
        prediction_provider=provider,
        cfg=bt_cfg,
        atr_series=atr_series,
        model_error_sigma_series=atr_series,
    )

    metrics = _compute_backtest_metrics(
        result,
        initial_equity=initial_equity,
        data=data,
    )

    row = {
        'final_equity': result.final_equity,
        'n_trades': len(result.trades),
        **metrics,
    }
    return result, row

# Grid A: risk_per_trade_pct × reward_risk_ratio

In [16]:
# Grid A: risk_per_trade_pct × reward_risk_ratio
risk_grid = [0.0025, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06]  # 0.0025, 0.005, 0.01, 0.03
rr_grid = [1.0, 1.5, 2.0, 3.0, 3.5] #[1.5, 2.0, 3.0]

rows = []
for risk_pct, rr in itertools.product(risk_grid, rr_grid):
    strat = StrategyConfig(
        risk_per_trade_pct=risk_pct,
        reward_risk_ratio=rr,
        k_sigma_err=0.5,
        k_atr_min_tp=3,
    )
    _, res_row = run_one(strat)
    res_row.updatea(
        {
            'grid': 'risk_rr',
            'risk_per_trade_pct': risk_pct,
            'reward_risk_ratio': rr,
            'k_sigma_err': strat.k_sigma_err,
            'k_atr_min_tp': strat.k_atr_min_tp,
        }
    )
    rows.append(res_row)

df_risk_rr = pd.DataFrame(rows).sort_values('sharpe_ratio', ascending=False)
df_risk_rr.head(10)

,final_equity,n_trades,total_return,cagr,max_drawdown,sharpe_ratio,win_rate,profit_factor,grid,risk_per_trade_pct,reward_risk_ratio,k_sigma_err,k_atr_min_tp
37,232320.996415,151,22.232100,1.976715,-0.509799,1.938318,0.476821,1.651881,risk_rr,0.060,2.0,0.5,3
32,149535.141853,151,13.953514,1.554947,-0.441680,1.938318,0.476821,1.673579,risk_rr,0.050,2.0,0.5,3
27,93170.493106,151,8.317049,1.168353,-0.366942,1.938311,0.476821,1.693997,risk_rr,0.040,2.0,0.5,3
22,56153.317159,151,4.615332,0.819161,-0.285425,1.938256,0.476821,1.713678,risk_rr,0.030,2.0,0.5,3
17,32705.228739,151,2.270523,0.508207,-0.197064,1.937755,0.476821,1.733694,risk_rr,0.020,2.0,0.5,3
12,18298.848415,151,0.829885,0.233117,-0.102605,1.920444,0.476821,1.746185,risk_rr,0.010,2.0,0.5,3
7,13417.859206,151,0.341786,0.107334,-0.054185,1.847378,0.476821,1.715889,risk_rr,0.005,2.0,0.5,3
36,89565.142259,119,7.956514,1.138880,-0.366587,1.766859,0.546218,1.543369,risk_rr,0.060,1.5,0.5,3
31,65161.086650,119,5.516109,0.915481,-0.313932,1.766845,0.546218,1.574212,risk_rr,0.050,1.5,0.5,3
26,46530.716965,119,3.653072,0.704360,-0.258025,1.766794,0.546218,1.607258,risk_rr,0.040,1.5,0.5,3


# Grid B: model trust (k_sigma_err) × noise filter (k_atr_min_tp)

In [14]:
# Grid B: model trust (k_sigma_err) × noise filter (k_atr_min_tp)
k_sigma_grid = [0.25, 0.5, 0.75, 1.0]
k_atr_grid = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0,]

rows = []
for k_sigma, k_atr_min_tp in itertools.product(k_sigma_grid, k_atr_grid):
    strat = StrategyConfig(
        risk_per_trade_pct=0.01,
        reward_risk_ratio=2.0,
        k_sigma_err=k_sigma,
        k_atr_min_tp=k_atr_min_tp,
    )
    _, res_row = run_one(strat)
    res_row.update(
        {
            'grid': 'noise_filters',
            'risk_per_trade_pct': strat.risk_per_trade_pct,
            'reward_risk_ratio': strat.reward_risk_ratio,
            'k_sigma_err': k_sigma,
            'k_atr_min_tp': k_atr_min_tp,
        }
    )
    rows.append(res_row)

df_noise = pd.DataFrame(rows).sort_values('sharpe_ratio', ascending=False)
df_noise.head(10)

,final_equity,n_trades,total_return,cagr,max_drawdown,sharpe_ratio,win_rate,profit_factor,grid,risk_per_trade_pct,reward_risk_ratio,k_sigma_err,k_atr_min_tp
11,18298.848415,151,0.829885,0.233117,-0.102605,1.920444,0.476821,1.746185,noise_filters,0.01,2.0,0.50,3.0
12,17661.463506,137,0.766146,0.218049,-0.089952,1.890635,0.481752,1.778099,noise_filters,0.01,2.0,0.50,3.5
10,18377.504314,168,0.837750,0.234953,-0.094474,1.852234,0.464286,1.665375,noise_filters,0.01,2.0,0.50,2.5
25,16864.052746,153,0.686405,0.198690,-0.081658,1.675680,0.457516,1.581276,noise_filters,0.01,2.0,1.00,3.0
19,16168.358888,128,0.616836,0.181305,-0.090215,1.668346,0.468750,1.696524,noise_filters,0.01,2.0,0.75,3.5
9,18128.927574,207,0.812893,0.229134,-0.118410,1.667974,0.439614,1.498542,noise_filters,0.01,2.0,0.50,2.0
20,15627.434490,111,0.562743,0.167447,-0.069262,1.653170,0.477477,1.757783,noise_filters,0.01,2.0,0.75,4.0
3,17423.284714,182,0.742328,0.212328,-0.080471,1.651029,0.445055,1.543218,noise_filters,0.01,2.0,0.25,2.5
4,16858.607770,159,0.685861,0.198555,-0.094246,1.649852,0.452830,1.581570,noise_filters,0.01,2.0,0.25,3.0
23,17612.659154,195,0.761266,0.216881,-0.101913,1.631994,0.441026,1.484972,noise_filters,0.01,2.0,1.00,2.0
